In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

from train_success import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os

In [5]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))

data_folder_path = os.path.join(root_dir, 'data')

file_path = os.path.join(data_folder_path, 'robotarm_2.csv')

df = pd.read_csv(file_path)

In [7]:
df.columns

Index(['Student ID', 'WorldSpace', 'index', 'Submission_TreeDist_Successive',
       'nb_tentative', 'pct_activity', 'error', 'missing', 'move1', 'move2',
       'pickup1', 'place1', 'nb_error', 'nb_missing', 'nb_move1', 'nb_move2',
       'nb_pickup1', 'nb_place1', 'pct_error', 'pct_missing', 'pct_move1',
       'pct_move2', 'pct_pickup1', 'pct_place1', 'euclidean_distance',
       'delta_successive', 'success', 'has_error', 'activity', 'seq_error',
       'case1', 'case2', 'case3', 'cases', 'activity_all',
       'bucket_delta_successive', 'bucket_Submission_TreeDist_Successive',
       'median_split_delta_successive',
       'median_split_Submission_TreeDist_Successive', 'at_least_2_tentatives',
       'error_sequence', 'case_sequence', 'success_seq'],
      dtype='object')

In [10]:
df['activity_all'].value_counts()

activity_all
123    3512
12     1199
1       370
13      181
23       76
2        44
3         5
Name: count, dtype: int64

In [11]:
df_123 = df[df['activity_all'] == 123]

In [13]:
df_123.to_csv(os.path.join(data_folder_path, 'robotarm_123.csv'), index=False)

In [157]:
df['success']

0       1
1       1
2       1
3       1
4       1
       ..
5382    0
5383    0
5384    0
5385    0
5386    0
Name: success, Length: 5387, dtype: int64

In [14]:
df_only_l1_l2 = df[df['activity'] != 1]

In [16]:
df_only_l1_l2.to_csv(os.path.join(data_folder_path, 'robotarm_only_l1_l2.csv'), index=False)

In [158]:
for idx, row in df.iterrows():
    df.at[idx, 'success_seq'] = 'success' if row['success'] == 1 else 'fail'

In [159]:
df.to_csv(file_path, index=False)

In [160]:
activities = [1, 1, 1]
split_type = 'distribution'

In [161]:
df_l1 = df[df['activity'] == 1]
df_l2 = df[df['activity'] == 2]
df_l3 = df[df['activity'] == 3]

In [204]:
world_sequences, code_sequences, error_sequences, case_sequences, success_seq = prepare_sequences(df, activities, split_type, one_hot=False)

In [205]:
len(world_sequences), len(code_sequences), len(error_sequences), len(case_sequences), len(success_seq)

(762, 762, 762, 762, 762)

In [206]:
train_size = 0.8

In [207]:
world_seq_train, world_seq_test, world_seq_valid = split_data(world_sequences, train_size)
code_seq_train, code_seq_test, code_seq_valid = split_data(code_sequences, train_size)
error_seq_train, error_seq_test, error_seq_valid = split_data(error_sequences, train_size)
case_seq_train, case_seq_test, case_seq_valid = split_data(case_sequences, train_size)
success_seq_train, success_seq_test, success_seq_valid = split_data(success_seq, train_size)

In [208]:
task = "error"
one_hot = False

In [209]:
X_train, y_train = create_features_labels(world_seq_train, code_seq_train, error_seq_train, case_seq_train, success_seq_train, task, one_hot)
X_test, y_test = create_features_labels(world_seq_test, code_seq_test, error_seq_test, case_seq_test, success_seq_test, task, one_hot)
X_valid, y_valid = create_features_labels(world_seq_valid, code_seq_valid, error_seq_valid, case_seq_valid, success_seq_valid, task, one_hot)

In [210]:
max_len = max(len(seq) for seq in X_train)
pad_value = [0, 1] if one_hot else [0]

In [211]:
X_train_padded = pad_with_pattern(X_train, max_len, pad_value)
X_test_padded = pad_with_pattern(X_test, max_len, pad_value)
X_valid_padded = pad_with_pattern(X_valid, max_len, pad_value)

In [212]:
X_train_padded

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [213]:
X_train = torch.tensor(X_train_padded, dtype=torch.float32)

In [216]:
args = {
    'model_type': 'rf',
    'data': df,
    'epochs': 100,
    'lr': 0.01,
    'dropout': 0.1,
    'weight_decay': 0,
    'hidden_dim': 100,
    'split_type': "distribution",
    'task': 'error', 
    'one_hot': False
}

In [217]:
cross_validate(**args, k=5)

TypeError: only integer scalar arrays can be converted to a scalar index